In [4]:
# Import library-library
import os
import glob
from tqdm import tqdm
import numpy as np

# Data Preparation and Preprocessing
import pandas as pd
import re
from string import digits

# Word Embedding
import joblib
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

# Input and Expansion Query
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import math
from textblob import TextBlob
from yake import KeywordExtractor
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk import tokenize
from operator import itemgetter
from nltk.tokenize import word_tokenize
from textblob import TextBlob
NLTK_StopWords = stopwords.words('indonesian')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/556 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if (((t.lower() == "tempat") or (t.lower() == "waktu")) or (t not in NLTK_StopWords))]
    return T

In [6]:
df_test = pd.read_csv('df_test.csv')
df_test = df_test[pd.notnull(df_test['description'])]
print(df_test.info())
print ('-'*90)
document_text_test= joblib.load('desc_text_test.pkl')
print(len(document_text_test))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181 entries, 0 to 180
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        181 non-null    object
 1   date         181 non-null    object
 2   description  181 non-null    object
 3   source       181 non-null    object
dtypes: object(4)
memory usage: 7.1+ KB
None
------------------------------------------------------------------------------------------
181


In [7]:
df_train = pd.read_csv('df_train.csv')
df_train = df_train[pd.notnull(df_train['description'])]
print(df_train.info())
print ('-'*90)
document_text_train= joblib.load('desc_text_train.pkl')
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1623 entries, 0 to 1622
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1623 non-null   object
 1   date         1623 non-null   object
 2   description  1623 non-null   object
 3   source       1623 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB
None
------------------------------------------------------------------------------------------
1623


In [10]:
bow_kecelakaan_read = pd.read_csv('bow_kriminalitas.csv')
# bow_kecelakaan_read.head()
bow_kecelakaan_text = []

for i in range(0, bow_kecelakaan_read.shape[0]):
    bow_kecelakaan_text.append(bow_kecelakaan_read.iloc[i,1])

In [11]:
bow_where_read = pd.read_csv('bow_where.csv')
bow_where_text = []

for i in range(0, bow_where_read.shape[0]):
    bow_where_text.append(bow_where_read.iloc[i,1])

In [12]:
bow_when_read = pd.read_csv('bow_when.csv')
bow_when_text = []

for i in range(0, bow_when_read.shape[0]):
    bow_when_text.append(bow_when_read.iloc[i,1])

In [13]:
bow_who_read = pd.read_csv('bow_who.csv')
bow_who_text = []

for i in range(0, bow_who_read.shape[0]):
    bow_who_text.append(bow_who_read.iloc[i,1])


In [14]:
def cari_dokpertama(kueriAsli):
    kueriPre=preprocessing(kueriAsli)
    kueriPre= " ".join (kueriPre)
    hasilSearch=[]
    tfidf_train = joblib.load('tfidf_train.pkl')
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    query_vec= tfidf_vectorizer.transform([kueriPre])
    # print('queryvec')
    print(query_vec)
    results=cosine_similarity(tfidf_train, query_vec).reshape((-1))
    # print(results)
    for i in results.argsort()[-5:][::-1]:
        hasilSearch.append(df_train.iloc[i,-2])
    hasilSearch=". ".join(hasilSearch)
    return hasilSearch

In [15]:
##Keywords Extraction with YAKE
def keyword_yake(hasilSearch):
    keywordYake=[]

    k_extractor = KeywordExtractor(lan="id", n=1, top=10)
    k_extractor2 = KeywordExtractor(lan="id", n=2, top=10)
    keywords = k_extractor.extract_keywords(text=hasilSearch)
    keywords = k_extractor2.extract_keywords(text=hasilSearch)
    keywordYake = [x for x, y in keywords]
    #keywordYake.append(keywords)
    #print (keywordYake)
    return keywordYake
#print("Keywords of article\n", keywords)


In [16]:
#Keywords Extraction with TFIDF
def keyword_tfidf(hasilSearch):

    keywordtfidf=[]
    keywordtfidf2=[]

    #doc = 'بَاب فرض الْوضُوء وسننه وهيآته وَفرض الْوضُوء سِتّ خِصَال النِّيَّة عمند غسل الْوَجْه وَغسل الْوَجْه وَغسل الذراعين مَعَ الْمرْفقين وَمسح مَا قل من الرَّأْس وَغسل الرجلَيْن مَعَ الْكَعْبَيْنِ وَالتَّرْتِيب وعَلى قَول الْوَلَاء وسننه عشر خِصَال خمس مِنْهَا قبل غسل الْوَجْه وَهِي التَّسْمِيَة وَغسل الْكَفَّيْنِ والمضمضة وَالِاسْتِنْشَاق وَالْمُبَالغَة فيههما إِلَّا للصَّائِم وَخمْس بعد غسل الْوَجْه وَهِي تَقْدِيم الْيُمْنَى على ليسرى وَمسح جَمِيع الرَّأْس وَمسح الْأُذُنَيْنِ ظاهرهما وباطنهما وَإِدْخَال الأصبعين فيهمَا وتخليل أَصَابِع الرجلَيْن . وَغسل دَاخل الْكَعْبَيْنِ وَلَيْسَ مسح لعنق من سنَنه وفضيلته تكراره ثَلَاثًا وزالواجب فِيهِ مرّة والمرتان أفضل وَالثَّلَاث أكمل وهيآته أَن يبْدَأ فِي تَطْهِير الْأَعْضَاء بمواضع الِابْتِدَاء . فَإِن اقْتصر على فروضه استة أَجزَأَهُ وَإِن ضيع حَظّ نَفسه فِيمَا ترك'
    total_words = re.sub(r'[^\w]', ' ', hasilSearch)
    total_words = total_words.lower().split()
    #print (total_words)
    total_word_length = len(total_words)
    total_sentences = tokenize.sent_tokenize(hasilSearch)
    total_sent_len = len(total_sentences)

    tf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))

    idf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

    #print(idf_score)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    def get_top_n(dict_elem, n):
        result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
        hasil =list(result.keys())
        #print(list(result.keys()))        
        return hasil
    #print(get_top_n(tf_idf_score, 25))
    #print(len(get_top_n(tf_idf_score, 1)))
    keywordtfidf.append(get_top_n(tf_idf_score, 25))
    for i in range(len(keywordtfidf)):
        #print (i)
        totalKw=0
        totalKw=len(keywordtfidf[i])
        for j in range(totalKw):
            #print (j)
            keywordtfidf2.append(keywordtfidf[i][j])
    #print (keywordtfidf2)
    return keywordtfidf2

In [17]:
#Keywords Extraction with BERT
def keyword_bert(hasilSearch):

    keywordbert=[]

    #for j in range(len(array_text)):
    keyword1 = kw_extractor.extract_keywords(hasilSearch, top_n=10, keyphrase_ngram_range=(1, 1))
    keyword2 = kw_extractor.extract_keywords(hasilSearch, top_n=10, keyphrase_ngram_range=(1, 2))

    #print("Keywords of article\n", keywords)
    for i in range (0,len (keyword1)):
        keywordbert.append(keyword1[i][0])
        keywordbert.append(keyword2[i][0])
    #print (keywordbert)
    return keywordbert

In [18]:
def rangking (keywordGabung):
    keywordTemp=[]
    keywordFinal=[]

    def borda_sort(lists):
        scores = {}
        for l in lists:
            for idx, elem in enumerate(reversed(l)):
                if not elem in scores:
                    scores[elem] = 0
                scores[elem] += idx
        return sorted(scores.keys(), key=lambda elem: scores[elem], reverse=True)

    keywordTemp.append(borda_sort(keywordGabung))
    print ('kandidat temp',keywordTemp)
    print ('Total Kandidat temp: ',len(keywordTemp[0]))

    if len(keywordTemp[0])>50:
        print ('kurang dari 50')
        for i in range (0,50):
            keywordFinal.append(keywordTemp[0][i])
    elif len(keywordTemp[0])<50:
        print ('lebih dari 50')
        for i in range (0,len(keywordTemp)):
            for j in range (0,len(keywordTemp[0])):
                keywordFinal.append(keywordTemp[0][j])
    print ('Total Kandidat final: ',len(keywordFinal))
    print ('Kandidat final: ',keywordFinal)
    return keywordFinal

In [19]:
def keyword_BOW(keywordBOW, kueriAsli):
    cekDuplicate = []
    kandidatFix = []

    for i in keywordBOW:
        if(i not in cekDuplicate and i!=0):
            cekDuplicate.append(i)

    queries=[kueriAsli]
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(cekDuplicate)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 10
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFix.append(cekDuplicate[idx])
    return kandidatFix

In [20]:
def kandidatFix(kueriAsli,bow):
    #kueri what
    # kueriAsli='apa sebenarnya kejadian yang terjadi diberita tersebut'
    kueri=preprocessing(kueriAsli)
    kueri= [" ".join (kueri)]
    print (kueri)
    hasilkandidat=[]
    keywordGabung=[]
    kandidatFix=[]
    kueriFix=[]
    


    hasilSearch=cari_dokpertama(kueriAsli)
    keywordYake=keyword_yake(hasilSearch)
    keywordtfidf2=keyword_tfidf(hasilSearch)
    keywordbert=keyword_bert (hasilSearch)
    keywordBOW=keyword_BOW(bow, kueriAsli)

    keywordGabung.append(keywordYake)
    keywordGabung.append(keywordtfidf2)
    keywordGabung.append(keywordbert)
    keywordGabung.append(keywordBOW)
    hasilrank=rangking(keywordGabung)

    for i in hasilrank:
        kueriFix.append(i)
    for j in kueriFix:
        hasilkandidat.append(j)
    kueriFix=[preprocessing(i) for i in kueriFix]
    for i in kueriFix:
        for j in i:
            kandidatFix.append(j)
    kandidatFix= [" ".join (kandidatFix)]
    
    print ('*'*120)
    return(kandidatFix,keywordGabung,keywordBOW,hasilrank)

In [21]:
hasilDokumenWhen=[]
hasilDokumenWho=[]
hasilDokumenWhere=[]
hasilDokumenWhat=[]

kueriAsliWhat='peristiwa apa yang terjadi pada berita tersebut'
kueriAsliWhere='di daerah mana peristiwa itu terjadi'
kueriAsliWho='siapa pelaku peristiwa tersebut'
kueriAsliWhen='Kapan waktu peristiwa tersebut'

kandidatFixWhat,keywordGabungWhat,keywordBOW_What, hasilrankWhat = kandidatFix(kueriAsliWhat, bow_kecelakaan_text)
kandidatFixWhere,keywordGabungWhere,keywordBOW_Where, hasilrankWhere = kandidatFix(kueriAsliWhere, bow_where_text)
kandidatFixWho,keywordGabungWho,keywordBOW_Who, hasilrankWho = kandidatFix(kueriAsliWho, bow_who_text)
kandidatFixWhen,keywordGabungWhen,keywordBOW_When, hasilrankWhen = kandidatFix(kueriAsliWhen, bow_when_text)

['peristiwa berita']
  (0, 10538)	0.4897637899726086
  (0, 1266)	0.8718551657423763
kandidat temp [['aksi', 'informasi', 'dihimpun', 'wib', 'diduga', 'aksinya', 'pembunuhan', 'sebilah', 'pembunuhan pemeriksaan', 'korbannya', 'diberitakan', 'tetangga', 'tersangka dilaporkan', 'aris', 'kejahatan', 'setiawan', 'kekuatan berwenang', 'saksi', 'pemeriksaan', 'mata', 'dibantu pelaku', 'ibunya', 'kekuatan', 'tetangganya', 'mengambil', 'puluh anggota', 'celurit membacok', 'mengungkap', 'terjadinya', 'membacok saudara', 'kabur', 'dibunuhnya dibantu', 'mempedulikan', 'dijumpainya aris', 'silakan', 'selengkapnya', 'kepentingannya', 'kantor berita', 'puluh', 'dipanggilnya diinterogasi', 'dipersalahkan', 'tetangganya membacok', 'geng', 'dipanggilnya', 'merebaknya', 'membacok kabur', 'menggerebek', 'selengkapnya remaja', 'timbulnya', 'kabur membacok', 'melibatkan', 'penyelidikan', 'rusaknya', 'dihimpun peristiwa', 'kota', 'terpaksa menangguhkan', 'pengungkapan', 'peristiwa wib', 'varginha', 'menyalah

In [22]:
# j=1

testing_data = []

for i in range(0, len(document_text_test)-1):

    hasilWhat=[]

    teks=df_test.iloc[i,-2]
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    tfidf_matrix = tfidf_vectorizer.fit_transform([teks])

    query_vec_What= tfidf_vectorizer.transform(kandidatFixWhat)
    results_what=cosine_similarity(tfidf_matrix, query_vec_What).reshape((-1))
    hasilDokumenWhat.append(df_test.iloc[i,2])
    for a in hasilrankWhat:
        cariW = re.findall(a,hasilDokumenWhat[i])
        #print(cariW)
        if cariW:
            hasilWhat.append(a)
    
    data = [i,df_test.iloc[i,2],'what',kueriAsliWhat,keywordBOW_What , keywordGabungWhat, hasilWhat, results_what,' ',' ']

    testing_data.append(data)

for i in range(0, len(document_text_test)-1):

    hasilWhere=[]

    query_vec_Where= tfidf_vectorizer.transform(kandidatFixWhere)
    results_where=cosine_similarity(tfidf_matrix, query_vec_Where).reshape((-1))
    hasilDokumenWhere.append(df_test.iloc[i,2])
    for a in hasilrankWhere:
        cariW = re.findall(a,hasilDokumenWhere[i])
        #print(cariW)
        if cariW:
            hasilWhere.append(a)
    
    data = [i,df_test.iloc[i,2],'where',kueriAsliWhere,keywordBOW_Where , keywordGabungWhere, hasilWhere, results_where,' ',' ']

    testing_data.append(data)

for i in range(0, len(document_text_test)-1):

    hasilWho=[]

    query_vec_Who= tfidf_vectorizer.transform(kandidatFixWho)
    results_who=cosine_similarity(tfidf_matrix, query_vec_Who).reshape((-1))
    hasilDokumenWho.append(df_test.iloc[i,2])
    for a in hasilrankWho:
        cariW = re.findall(a,hasilDokumenWho[i])
        #print(cariW)
        if cariW:
            hasilWho.append(a)
    
    data = [i,df_test.iloc[i,2],'who',kueriAsliWho,keywordBOW_Who , keywordGabungWho, hasilWho, results_who,' ',' ']

    testing_data.append(data)

for i in range(0, len(document_text_test)-1):

    hasilWhen=[]

    query_vec_When= tfidf_vectorizer.transform(kandidatFixWhen)
    results_when=cosine_similarity(tfidf_matrix, query_vec_When).reshape((-1))
    hasilDokumenWhen.append(df_test.iloc[i,2])
    for a in hasilrankWhen:
        cariW = re.findall(a,hasilDokumenWhen[i])
        #print(cariW)
        if cariW:
            hasilWhen.append(a)
    
    data = [i,df_test.iloc[i,2],'when',kueriAsliWhen,keywordBOW_When , keywordGabungWhen, hasilWhen, results_when,' ',' ']

    testing_data.append(data)

    # print (j)
    # print("No ID Dokumen  : ", i)
    # print("Tanggal        : ", df_test.iloc[i,1])
    # print("Isi berita     : ", df_test.iloc[i,2])
    # print ("Hasil W       : ",hasilWhat)
    # print("(Kemiripan: %.4f) " % results_what)
    # print ('*'*120)
    # j+=1

In [23]:
#write to csv
writer = pd.DataFrame(testing_data, columns=['No Document','Description', 'W','Keyword W', 'Keyword BOW', 'Keyword Gabung','hasilW', 'Kemiripan', 'True Positif', 'True Negative'])
writer.to_csv('QE-Statistik-result.csv', index=False, sep=',')

In [24]:
QE_result_read = pd.read_csv('QE-Statistik-result.csv')
QE_result_read.head()

,No Document,Description,W,Keyword W,Keyword BOW,Keyword Gabung,hasilW,Kemiripan,True Positif,True Negative
0,0,polisi mengungkap beredarnya rekaman tim gabun...,what,peristiwa apa yang terjadi pada berita tersebut,"['terjadinya', 'mempedulikan', 'kepentingannya...","[['celurit membacok', 'membacok saudara', 'dij...",['mengungkap'],[0.17334381],,
1,1,polda metro jaya mengungkap penembakan peristi...,what,peristiwa apa yang terjadi pada berita tersebut,"['terjadinya', 'mempedulikan', 'kepentingannya...","[['celurit membacok', 'membacok saudara', 'dij...","['wib', 'mengungkap']",[0.17609018],,
2,2,polisi menangkap pria tempat tersangka pencuri...,what,peristiwa apa yang terjadi pada berita tersebut,"['terjadinya', 'mempedulikan', 'kepentingannya...","[['celurit membacok', 'membacok saudara', 'dij...","['aksi', 'korbannya']",[0.41866242],,
3,3,kepala smpn berinisial sa suwata sa teledor vi...,what,peristiwa apa yang terjadi pada berita tersebut,"['terjadinya', 'mempedulikan', 'kepentingannya...","[['celurit membacok', 'membacok saudara', 'dij...",['diberitakan'],[0.08720414],,
4,4,polisi menetapkan penyanyi dangdut alasannya m...,what,peristiwa apa yang terjadi pada berita tersebut,"['terjadinya', 'mempedulikan', 'kepentingannya...","[['celurit membacok', 'membacok saudara', 'dij...",[],[0.10259784],,
